# Load packages

In [ ]:
import pandas as pd
import numpy as np
from datetime import date
from IPython.display import Markdown, display
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# (auxilliary function for printing markdown)
def printmd(string):
    display(Markdown(string))

# Load event log

In [ ]:
file = 'ArtificialPatientTreatment.csv'
# row = event about performing *activity* at *timestamp* concerning *patient case*
# (here, also incl. the *resource* involved; the consulting doctor)
events = pd.read_csv(file)

# also do some initial data prep
events.columns = ['patient', 'action', 'resource', 'timestamp']
events['timestamp'] = pd.to_datetime(events['timestamp'])
events['action'] = events['action'].apply(lambda x: x.strip())

# print out first few rows to get an idea of the data
events.head()


In [ ]:
print('{} has {} rows and {} columns.'.format(file, events.shape[0], events.shape[1]))

# Log preparation

In [ ]:
# - per case, get the start and end times

# (start & end times of patient cases =  min. and max. value of all timestamps per patient, respectively)
case_starts_ends = events.pivot_table(index='patient', aggfunc={'timestamp': ['min', 'max']})
case_starts_ends = case_starts_ends.reset_index()
case_starts_ends.columns = ['patient', 'caseend', 'casestart']
# (add these new columns to event table)
events = events.merge(case_starts_ends, on='patient')


# - per event, get relative time since start of case

events['relativetime'] = events['timestamp'] - events['casestart']
events.head()

# Extend event log with extra attributes

In [ ]:
# - per case, add attributes to event log:
# (1) single string with sequence of activities (action_sequence)
# (2) count total number of events (numactions)

delimiter = '___'

makeEventString = lambda x: delimiter.join(x)
makeEventString.__name__ = 'makeEventString'

numEvents = lambda x: len(x)
numEvents.__name__ = 'numEvents'

caselogs = events.pivot_table(index='patient', aggfunc={'action': [makeEventString, numEvents]})
caselogs = caselogs.reset_index()
caselogs.columns = ['patient', 'action_sequence', 'numactions']

events = pd.merge(events, caselogs, on='patient')
events['caselength'] = events['caseend'] - events['casestart']

events.head()


In [ ]:
# - per event, add weekday, date & hour of timestamp; date of start-time; seconds & days of relative time

events['weekday'] = events['timestamp'].apply(lambda x: x.weekday())
events['date'] = events['timestamp'].apply(lambda x: x.date())
events['hour'] = events['timestamp'].apply(lambda x: x.time().hour)
events['startdate'] = events['casestart'].apply(lambda x: x.date())
## Get relative times in more friendly terms
events['relativetime_s'] = events['relativetime'].dt.seconds + 86400*events['relativetime'].dt.days
events['relativedays'] = events['relativetime'].dt.days

events.head()

# Questions
Ask some questions about the dataeset.

## What is the minimum number of events per case?

In [ ]:
printmd('**Minimum number of events per case**: {}'.format(min(events['patient'].value_counts())))

## Patient 26
* Which doctor did they have their first & last consultation with?

In [ ]:
first_doctor = events[events['timestamp']==min(events[events['patient']=='patient 26']['timestamp'])]['resource'].values[0]
last_doctor = events[events['timestamp']==max(events[events['patient']=='patient 26']['timestamp'])]['resource'].values[0]
printmd('**First doctor**: {}'.format(first_doctor))
printmd('**Last doctor**: {}'.format(last_doctor))

## Which activity has the lowest occurrence overall in the event log?

In [ ]:
printmd('**Activity with lowest occurrence**: {}'.format(events['action'].value_counts().sort_values().idxmin()))

# Visualisations

In [ ]:
# - select all events belonging to first 50 patients

patients = events['patient'].unique()
selected_patients = patients[0:50]
patientX = events[events['patient'].isin(selected_patients)]

In [ ]:
# - prepare some lists for visualization

# per event, get patient case ids (used to setup Y axis ticks)
patientnums = [int(e) for e in events['patient'].apply(lambda x: x.strip('patient'))]
# per event, get ordinal for resource (used in later code)
resourcenums = [i for (i, e) in enumerate(events['resource'])]

## Simple scatter plots

### Date

In [ ]:
# - plot resource usage over time

ax = sns.scatterplot(x=events['timestamp'], y=events['resource'], hue=events['action'])

### Relative time 
Time since start of case.

* y-axis represents each patient case.
* x-axis represents time since case was initiated.
* Different marker colors represent different actions.

In [ ]:
# - per patient case, time in days per activity

ax = sns.scatterplot(x=events['relativedays'], y=events['patient'], hue=events['action'])
plt.yticks(np.arange(min(patientnums), max(patientnums)+1, 5));


In [ ]:
# - sort events by case length and relative time to start

ordered = events.sort_values(by=['caselength', 'relativedays'])

In [ ]:
# - now, per patient case, time in days per activity (sorted events)

ax = sns.scatterplot(x=ordered['relativedays'], y=ordered['patient'], hue=ordered['action'])
plt.yticks(np.arange(min(patientnums), max(patientnums)+1, 5));
plt.show()


# Get case data per patient (interactive)

In [ ]:
# (widget libraries)
from ipywidgets import widgets
from ipywidgets import interact, interact_manual

patients = events['patient'].unique()

@interact
def getCaseData(x=patients):
    return events[events['patient']==x]

# Filtering events

## Getting events that are (not) common to all patients

In [ ]:
# get frequency table (rows = patient cases; columns = actions; values = number of times action occurs for case)
patient_events = pd.crosstab(events['patient'], events['action'])

# create heat map
sns.heatmap(patient_events, cmap="YlGnBu")

# per column (action), get number of unique values
nunique = patient_events.apply(pd.Series.nunique)

common_actions = nunique[nunique==1].index # (means only 1's are found)
uncommon_actions = nunique[nunique>1].index # (means both 0's and 1's are found)
printmd('**The following actions are common to all cases**: {}'.format(', '.join(common_actions)))
printmd('**The following actions are not common to all cases**: {}'.format(', '.join(uncommon_actions)))


## Filter events to only include uncommon events

In [ ]:
filtered = events[events['action'].isin(uncommon_actions)]
patient_events = pd.crosstab(filtered['patient'], filtered['action'])

printmd('**The filtered data has** {} **rows and** {} **columns.**'.format(filtered.shape[0], filtered.shape[1]))
printmd('**This amounts to** {} **cases with** {} **distinct actions.**'.format(patient_events.shape[0], patient_events.shape[1]))

# minus x-ray scans?
filtered = filtered[filtered['action']!='X-ray scan']
printmd('**The filtered data excluding X-rays has** {} **rows and** {} **columns.**'.format(filtered.shape[0], filtered.shape[1]))

# filtered heat map
sns.heatmap(patient_events, cmap="YlGnBu")

# Process Mining
* Check out this [introduction to process mining in Python](https://towardsdatascience.com/introduction-to-process-mining-5f4ce985b7e5).
* [Documentation for pm4py](https://pm4py.fit.fraunhofer.de/)

## Load packages

In [ ]:
# run the following in terminal:
# %> pip install pm4py

import pm4py

# converters
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.importer.xes import importer as xes_importer

# process mining 
from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.algo.filtering.dfg.dfg_filtering import clean_dfg_based_on_noise_thresh

# social network analysis
from pm4py.algo.organizational_mining.sna import algorithm as sna_algorithm
from pm4py.visualization.sna import visualizer as pn_vis

# visualization
# (wvw: updated, courtesy https://stackoverflow.com/questions/75424412/no-module-named-pm4py-objects-petri-in-pm4py)
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.visualization.petri_net import visualizer as pn_visualizer
# (wvw: added)
from pm4py.visualization.dfg import visualizer as dfg_visualizer
from pm4py.visualization.heuristics_net import visualizer as hn_visualizer
from pm4py.visualization.process_tree import visualizer as pt_visualizer

# misc 
from pm4py.objects.conversion.process_tree import converter as pt_converter
from pm4py.visualization.petri_net.util import performance_map 

## Log preparation

In [ ]:
eventlog = events.copy()

# rename columns in accordance with pm4py
# specify columns corresponding to case (case:concept:name), event (concept:name) & timestamp (time:timestamp)

eventlog.rename(columns={'timestamp': 'time:timestamp', 'patient': 'case:concept:name', 'action': 'concept:name', 'resource': 'org:resource'}, inplace=True)

# convert to log format
log = log_converter.apply(eventlog)

eventlog.head()

## Process mining algorithms

### Directly-follows graph

In [ ]:
# create graph from log
dfg = dfg_discovery.apply(log, variant=dfg_discovery.Variants.PERFORMANCE)

# visualize performance (durations)
gviz = dfg_visualizer.apply(dfg, log=log, variant=dfg_visualizer.Variants.PERFORMANCE)
dfg_visualizer.view(gviz)

# admire the spaghetti ...

With average times between nodes (performance)

In [ ]:
# create graph from log
dfg = dfg_discovery.apply(log, variant=dfg_discovery.Variants.FREQUENCY)

# visualize performance (frequency)

gviz = dfg_visualizer.apply(dfg, log=log, variant=dfg_visualizer.Variants.FREQUENCY)
dfg_visualizer.view(gviz)

### Alpha miner

In [ ]:
# alpha miner
net, initial_marking, final_marking = alpha_miner.apply(log)

# visualise
gviz = pn_visualizer.apply(net, initial_marking, final_marking)
pn_visualizer.view(gviz)

# better!

In [ ]:
# add frequency to visualization
gviz = pn_visualizer.apply(net, initial_marking, final_marking, 
                           variant=pn_visualizer.Variants.FREQUENCY, 
                           log=log)

pn_visualizer.view(gviz)

# (save the Petri net)
# pn_visualizer.save(gviz, "alpha_miner_healthcare_petri_net.png")


# compare with heat map with uncommon events:
# (medicine is slightly less "popular" than x-rays, and surgery is least popular)

### Heuristic miner

In [ ]:
# heuristics miner
heu_net = heuristics_miner.apply_heu(log)

# visualize
gviz = hn_visualizer.apply(heu_net)
hn_visualizer.view(gviz)

In [ ]:
# show petri net (same output as alpha miner), incl. frequencies

# heuristics miner
net, im, fm = heuristics_miner.apply(log)

# visualize
gviz = pn_visualizer.apply(net, im, fm, 
                           variant=pn_visualizer.Variants.FREQUENCY, 
                           log=log)
pn_visualizer.view(gviz)

### Inductive miner

In [ ]:
# create the process tree
# (wvw: drop "_tree" from call)
tree = inductive_miner.apply(log)

# visualize
gviz = pt_visualizer.apply(tree)
pt_visualizer.view(gviz)

# creates a process treee instead of petri net

In [ ]:
# show petri net, incl. frequencies

# (wvw - only process tree mining seems to be supported currently)

# convert the process tree to a petri net
net, initial_marking, final_marking = pt_converter.apply(tree)


# (wvw: not working for me)
# alternatively, use the inductive_miner to create a petri net from scratch
# net, initial_marking, final_marking = inductive_miner.apply(log)

# viz
# parameters = {pn_visualizer.Variants.FREQUENCY.value.Parameters.FORMAT: "png"}
gviz = pn_visualizer.apply(net, initial_marking, final_marking, 
                        #    parameters=parameters, 
                           variant=pn_visualizer.Variants.FREQUENCY, 
                           log=log)
pn_visualizer.view(gviz)

# pn_visualizer.save(gviz, "inductive_miner_healthcare_petri_net.png")

## Variants

Get unique process variants, i.e., unique sequences of activities.

In [ ]:
variants = pm4py.get_variants(log)
variants = pd.DataFrame(variants.items())

variants

## Social Networks

* See [this guide](https://pm4py.fit.fraunhofer.de/documentation#social-network-analysis) on using pm4py to analyse social networks

In [ ]:
# different types of social interactions
hw_values = sna_algorithm.apply(log, variant=sna_algorithm.Variants.HANDOVER_LOG)
wt_values = sna_algorithm.apply(log, variant=sna_algorithm.Variants.WORKING_TOGETHER_LOG)
sub_values = sna_algorithm.apply(log, variant=sna_algorithm.Variants.SUBCONTRACTING_LOG)
ja_values = sna_algorithm.apply(log, variant=sna_algorithm.Variants.JOINTACTIVITIES_LOG)

gviz_hw = pn_vis.apply(hw_values, variant=pn_vis.Variants.NETWORKX,
                       parameters={pn_vis.Variants.NETWORKX.value.Parameters.FORMAT: "png"})
gviz_wt = pn_vis.apply(wt_values, variant=pn_vis.Variants.NETWORKX,
                       parameters={pn_vis.Variants.NETWORKX.value.Parameters.FORMAT: "png"})
gviz_sub = pn_vis.apply(sub_values, variant=pn_vis.Variants.NETWORKX,
                        parameters={pn_vis.Variants.NETWORKX.value.Parameters.FORMAT: "png"})
gviz_ja = pn_vis.apply(ja_values, variant=pn_vis.Variants.NETWORKX,
                       parameters={pn_vis.Variants.NETWORKX.value.Parameters.FORMAT: "png"})

pn_vis.view(gviz_hw, variant=pn_vis.Variants.NETWORKX)
pn_vis.view(gviz_wt, variant=pn_vis.Variants.NETWORKX)
pn_vis.view(gviz_sub, variant=pn_vis.Variants.NETWORKX)
pn_vis.view(gviz_ja, variant=pn_vis.Variants.NETWORKX)

# Extra material

I did not find the code below very useful, including it here for completeness

## Simple scatter plots


In [ ]:
ax = sns.scatterplot(x=events['timestamp'], y=events['patient'], hue=events['action'])
plt.yticks(np.arange(min(patientnums), max(patientnums)+1, 5));

In [ ]:
# - per resource, time in days per activity (sorted events)

ax = sns.scatterplot(x=ordered['relativedays'], y=ordered['resource'], hue=ordered['action'])
plt.show()


In [ ]:
pd.crosstab(events['action'], events['resource'], normalize='columns')

## Simple stripplots

### Weekday

In [ ]:
# - Y axis = patient ordinal; X axis = weekday; dots = action

ax = sns.stripplot(x=events['weekday'], y=patientnums, hue=events['action'], jitter=0.2)
#plt.yticks(np.arange(min(patientnums), max(patientnums)+1, 5));

In [ ]:
# - Y axis = resource ordinal; X axis = weekday; dots = action

ax = sns.stripplot(x=events['weekday'], y=resourcenums, hue=events['action'], jitter=0.2)

### Hour

In [ ]:
# - Y axis = patient ordinal; X axis = hour; dots = action

ax = sns.stripplot(x=events['hour'], y=patientnums, hue=events['action'], jitter=0.2)


In [ ]:
# - Y axis = resource ordinal; X axis = hour; dots = action

ax = sns.stripplot(x=events['hour'], y=resourcenums, hue=events['action'], jitter=0.2)

In [ ]:
# - use unique markers per activity

activities = list(events['action'].unique())
markers = ['*', '+', 'h', 'o', 'x', 'D', '^', 'v']

# (same # of markers as # of activities)
assert(len(activities)==len(markers))

def getEventPlot(patientlist=[patients[10], patients[21]]):
    fig, ax = plt.subplots(figsize=(20, 5))
    
    for x in patientlist:
        patientX = getCaseData(x)
        
    
        for i in range(0, len(activities)):
            a = activities[i]
            marker = markers[i]
            selected = patientX[patientX['action']==a]
            ax.plot(selected['relativetime_s'], 
                selected['patient'],
                marker=marker, markersize=11,
                alpha=0.9, color='blue', linewidth=0, 
                label=a);
    plt.show()

getEventPlot()